In [1]:
import os
import sys
# 如果当前代码文件运行测试需要加入修改路径，避免出现后导包问题
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))

PYSPARK_PYTHON = "/miniconda2/envs/py365/bin/python"
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON

from offline import SparkSessionBase
import pyhdfs
import time


class UpdateUserProfile(SparkSessionBase):
    """离线相关处理程序
    """
    SPARK_APP_NAME = "updateUser"
    ENABLE_HIVE_SUPPORT = True
    SPARK_URL = "local"
#     SPARK_EXECUTOR_MEMORY = "2g"

    def __init__(self):

        self.spark = self._create_spark_session()

In [2]:
#初始化一个hive环境
uup = UpdateUserProfile()

In [3]:
uup.spark.sql('use profile')

DataFrame[]

In [6]:
# 读取日志数据，（关联历史日志数据和HIVE表分区）-----这里已经搞完毕了，再搞一次没关系
import pandas as pd
from datetime import datetime

def datelist(startdate, enddate):
    date = [datetime.strftime(d, '%Y-%m-%d') for d in list(pd.date_range(start=startdate, end=enddate))]
    return date

dl = datelist('2019-03-05', time.strftime('%Y-%m-%d', time.localtime()))


pydfs = pyhdfs.HdfsClient(hosts="hadoop-master:50070")
# 循环每个日期进行关联
for d in dl:
    
    # 构造hadoop地址
    _location = '/user/hive/warehouse/profile.db/user_action/' + d
    try:
        if pydfs.exists(_location):
            uup.spark.sql("alter table user_action add partition (dt='%s') location '%s'" % (d, _location))
    except:
        pass
        
    

In [15]:
# 如果hadoop没有今天该日期文件，则没有日志数据，结束----这里也已经搞完毕了
time_str = time.strftime("%Y-%m-%d", time.localtime())

_localions = '/user/hive/warehouse/profile.db/user_action/' + time_str
if pydfs.exists(_localions):
    # 如果有该文件直接关联，捕获关联重复异常
    try:
        uup.spark.sql("alter table user_action add partition (dt='%s') location '%s'" % (time_str, _localions))
    except Exception as e:
        pass

else:
    pass

In [4]:
#正常是每天增量更新，但是我们没有，只有历史数据，就从4月1号
# , param.articleId, param.algorithmCombine, param.action, param.userId
sqlDF = uup.spark.sql("select actionTime, readTime, channelId , param['articleId'] as articleId, param['algorithmCombine'] as algorithmCombine, param['action'] as action, param['userId'] as userId from user_action where dt='2019-03-06'")

In [6]:
sqlDF.show(truncate=False)

+-------------------+--------+---------+-----------------------------------------------------------------------+----------------+--------+------+
|actionTime         |readTime|channelId|articleId                                                              |algorithmCombine|action  |userId|
+-------------------+--------+---------+-----------------------------------------------------------------------+----------------+--------+------+
|2019-03-06 10:05:31|        |0        |[15140, 14883, 44451, 13476, 12971, 18220, 16045, 44466, 44597, 13750] |C2              |exposure|10    |
|2019-03-06 10:06:08|        |0        |[44348, 15549, 17215, 15680, 15808, 14529, 140357, 19016, 15048, 14668]|C2              |exposure|10    |
|2019-03-06 10:06:12|        |0        |[16334, 13648, 13778, 17748, 17364, 18135, 44376, 14681, 13401, 44000] |C2              |exposure|10    |
|2019-03-06 10:06:13|        |0        |[17120, 17506, 19171, 15716, 18806, 17784, 17657]                      |C2          

In [5]:
#上面一个用户对应多个文章
# [row.userId, row.actionTime, article_id, row.channelId, False, False, False, True, row.readTime]
def _compute(row):
    
    _list = []
    # 处理曝光日志行为
    if row.action == "exposure":
        # 处理其中的曝光推荐文章列表，编程一个用户对应一篇文章的数据
        for article_id in eval(row.articleId):
            # 返回多行结果
            _list.append([row.userId, row.actionTime, article_id, row.channelId, False, False, False, True, row.readTime])
    else:
        class Temp(object):
            shared = False
            clicked = False
            collected = False
            read_time = ""
        
        _tp = Temp()
        if row.action == "click":
            _tp.clicked = True
        elif row.action == "read":
            _tp.clicked = True
        elif row.action == "share":
            _tp.shared = True
        elif row.action == "collect":
            _tp.collected = True
        else:
            pass
        
        _list.append([row.userId, row.actionTime, int(row.articleId), row.channelId, _tp.shared, _tp.clicked, _tp.collected, True, row.readTime]) 
    return _list
    

_res = sqlDF.rdd.flatMap(_compute)


In [6]:
data = _res.toDF(["user_id", "action_time","article_id", "channel_id", "shared", "clicked", "collected", "exposure", "read_time"])



In [10]:
data.show()

+-------+-------------------+----------+----------+------+-------+---------+--------+---------+
|user_id|        action_time|article_id|channel_id|shared|clicked|collected|exposure|read_time|
+-------+-------------------+----------+----------+------+-------+---------+--------+---------+
|     10|2019-03-06 10:05:31|     15140|         0| false|  false|    false|    true|         |
|     10|2019-03-06 10:05:31|     14883|         0| false|  false|    false|    true|         |
|     10|2019-03-06 10:05:31|     44451|         0| false|  false|    false|    true|         |
|     10|2019-03-06 10:05:31|     13476|         0| false|  false|    false|    true|         |
|     10|2019-03-06 10:05:31|     12971|         0| false|  false|    false|    true|         |
|     10|2019-03-06 10:05:31|     18220|         0| false|  false|    false|    true|         |
|     10|2019-03-06 10:05:31|     16045|         0| false|  false|    false|    true|         |
|     10|2019-03-06 10:05:31|     44466|

In [7]:
# 将所有的某用户对于谋篇文章的操作合并到一行结果，因为原来浏览某篇文章，现在点击某一篇文章
# 首先合并历史行为数据
old = uup.spark.sql('select * from user_article_basic')


In [8]:
new_data = old.unionAll(data)

In [12]:
#为啥频道id为0，因为推荐给前端的时候，我们只给了文章id，没有给频道id，因此日志里记录的为0，我们分析日志时，要再次对应上
new_data.show()

+-------------------+-------------------+----------+----------+------+-------+---------+--------+---------+
|            user_id|        action_time|article_id|channel_id|shared|clicked|collected|exposure|read_time|
+-------------------+-------------------+----------+----------+------+-------+---------+--------+---------+
|1105045287866466304|2019-03-11 18:13:45|     14225|         0| false|  false|    false|    true|         |
|1106476833370537984|2019-03-15 16:46:50|     14208|         0| false|  false|    false|    true|         |
|1111189494544990208|2019-03-28 17:02:35|     19322|         0| false|  false|    false|    true|         |
|1111524501104885760|2019-03-29 15:04:27|     44161|         0| false|  false|    false|    true|         |
|1112727762809913344|2019-04-03 12:51:57|     18172|        18| false|   true|     true|    true|    19413|
|                  1|2019-03-07 16:57:34|     44386|        18| false|   true|    false|    true|    17850|
|                  1|2019-03

In [9]:
new_data.count()

2859

In [ ]:
#因为不支持原子操作，所以我们把新生成的数据都放进去
new_data.registerTempTable("temptable")
# 按照用户，文章分组存放进去----这里不执行，因为数据我已经插入好了
uup.spark.sql(
        "insert overwrite table user_article_basic select user_id, max(action_time) as action_time, "
        "article_id, max(channel_id) as channel_id, max(shared) as shared, max(clicked) as clicked, "
        "max(collected) as collected, max(exposure) as exposure, max(read_time) as read_time from temptable "
        "group by user_id, article_id")

In [10]:
# 1、读取user_article_basic表，合并行为表与文章画像中的主题词
# 离线所有用户行为都进行分析
uup.spark.sql("use profile")
# 对于日志当中的频道号，不是真正文章的对应频道，因为有0号频道存在
user_article_basic = uup.spark.sql("select * from user_article_basic").drop('channel_id')

In [11]:
# 读取文章画像, 打到用户身上的都是主题词
uup.spark.sql("use article")
article_profile = uup.spark.sql("select article_id, channel_id, topics from article_profile")

In [12]:
user_topics_list = user_article_basic.join(article_profile, how='left', on=['article_id'])

In [13]:
user_topics_list.show()

+-------------------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------+
|         article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|              topics|
+-------------------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------+
|              13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|              13401|1106396183141548032|2019-03-28 10:58:20| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|              14805|1105045287866466304|2019-03-11 18:15:48| false|  false|    false|    true|         |        18|[占位符, Code, sep, ...|
|              14805|                  1|2019-03-05 17:34:03| false|  false|    false|    true|         |        18|[占位符, Code, sep, ...|
|              14805|1111524501104

In [14]:
# 对于主题词列表展开，方便计算每个用户的关键词的权重
import pyspark.sql.functions as F
user_single_topic = user_topics_list.withColumn('topic', F.explode('topics')).drop('topics')

In [15]:
user_single_topic.show()

+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------+
|article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|   topic|
+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------+
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      补码|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|     字符串|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      李白|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|    type|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      元素|
|     13401|                 10|2019-03-06 10:06:12| false|  fal

In [16]:
user_single_topic.printSchema()

root
 |-- article_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- action_time: string (nullable = true)
 |-- shared: boolean (nullable = true)
 |-- clicked: boolean (nullable = true)
 |-- collected: boolean (nullable = true)
 |-- exposure: boolean (nullable = true)
 |-- read_time: string (nullable = true)
 |-- channel_id: integer (nullable = true)
 |-- topic: string (nullable = true)



In [18]:
def _save_weights(partition):
    """计算用户某个频道的某个词权重并落地
    """
    weightsOfaction = {
        "read_min": 1,
        "read_middle": 2,
        "collect": 2,
        "share": 3,
        "click": 5
    }
    
    import happybase
    from datetime import datetime
    import numpy as np
    import json
    pool = happybase.ConnectionPool(size=10, host='192.168.19.137', port=9090)
    for row in partition:
        # 计算时间衰减系数
#         t = datetime.now() - datetime.strptime(row.action_time, '%Y-%m-%d %H:%M:%S')
        t = datetime.strptime('2019-04-10 10:06:12', '%Y-%m-%d %H:%M:%S') - datetime.strptime(row.action_time, '%Y-%m-%d %H:%M:%S')
        # 间隔，取出t.days
        # 衰减系数公式
        time_expr = 1 / (np.log(t.days + 1) + 1)

        # 合并各种行为系数
        if row.read_time == "":
            read = 0
        else:
            read = int(row.read_time)

        # 取出这个时间对应的行为类型权重
        read_score = weightsOfaction['read_middle'] if read > 1000 else weightsOfaction['read_min']

        # 计算该词的权重row.topic
        weights = (read_score + row.collected * weightsOfaction['collect'] + row.shared * weightsOfaction['share'] + 
                 row.clicked * weightsOfaction['click']) * time_expr

        # 将 用户的频道的关键词以及权重值写入到hbase当中
        with pool.connection() as conn:
            table = conn.table('user_profile')
            table.put('user:{}'.format(row.user_id).encode(),
                     {'partial:{}:{}'.format(row.channel_id, row.topic).encode(): json.dumps(
                         weights).encode()})
            conn.close()
    

user_single_topic.foreachPartition(_save_weights)

In [24]:
import happybase
#  用于读取hbase缓存结果配置
pool = happybase.ConnectionPool(size=10, host='192.168.19.137', port=9090)

with pool.connection() as conn:
    table = conn.table('user_profile')
    # 获取每个键 对应的所有列的结果
    data = table.row(b'user:2', columns=[b'partial'])
    print(data)
    conn.close()

{b'partial:13:\xe4\xba\xba\xe5\xb7\xa5\xe6\x99\xba\xe8\x83\xbd': b'1.6439270311265748', b'partial:13:\xe4\xba\xba\xe7\xb1\xbb': b'1.6439270311265748', b'partial:13:\xe5\x88\x86\xe6\x94\xaf': b'1.6439270311265748', b'partial:13:\xe5\xad\xa6\xe7\xa7\x91': b'1.6439270311265748', b'partial:13:\xe5\xb0\x96\xe7\xab\xaf\xe6\x8a\x80\xe6\x9c\xaf': b'1.6439270311265748', b'partial:13:\xe6\x95\x99\xe6\x8e\x88': b'1.6439270311265748', b'partial:13:\xe6\x99\xba\xe5\x8a\x9b': b'1.6439270311265748', b'partial:13:\xe6\x99\xba\xe8\x83\xbd': b'1.6439270311265748', b'partial:13:ldquo': b'1.6439270311265748', b'partial:18:\xe4\xb8\x96\xe7\x95\x8c': b'1.3531838779170973', b'partial:18:\xe4\xb8\xaa\xe6\x95\xb0': b'1.2796234278822656', b'partial:18:\xe4\xb8\xad\xe6\x8b\xac\xe5\x8f\xb7': b'0.21107423703435377', b'partial:18:\xe4\xbb\xa3\xe6\x8c\x87': b'2.3761756624670785', b'partial:18:\xe4\xbb\xa3\xe7\xa0\x81\xe8\x87\xaa\xe5\x8a\xa8\xe8\xa1\xa5\xe5\x85\xa8': b'1.50099870681658', b'partial:18:\xe4\xbc\x9a\xe5

In [19]:
uup.spark.sql("use toutiao")
user_basic = uup.spark.sql("select user_id, gender, birthday from user_profile")


In [20]:
user_basic.show()

+-------------------+------+----------+
|            user_id|gender|  birthday|
+-------------------+------+----------+
|                  1| false|      null|
|                 10| false|      null|
|                 11| false|      null|
|1102490522829717504| false|      null|
|1102561962748805120| false|      null|
|1103195673450250240| false|      null|
|1105045287866466304| false|      null|
|1105093883106164736| false|      null|
|1105105185656537088| false|      null|
|1106396183141548032| false|      null|
|1106473203766657024| false|      null|
|1106476833370537984| false|      null|
|1114863846486441984| false|1997-01-21|
|1114863941936218112|  true|      null|
|1114863991156375552| false|      null|
|1114863998437687296| false|      null|
|1114871412419461120|  true|      null|
|1115436666438287360| false|      null|
|1115504747848138752| false|      null|
|1115534631668547584| false|      null|
+-------------------+------+----------+
only showing top 20 rows



In [25]:

# 更新用户基础信息
def _udapte_user_basic(partition):
    """更新用户基本信息
    """
    import happybase
    import json
    from datetime import datetime
    #  用于读取hbase缓存结果配置
    pool = happybase.ConnectionPool(size=10, host='192.168.19.137', port=9090)
    for row in partition:

        from datetime import date
        age = 0
        if row.birthday != 'null':
            born = datetime.strptime(row.birthday, '%Y-%m-%d')
            today = date.today()
            age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))  #年龄是今天减去出生日期

        with pool.connection() as conn:
            table = conn.table('user_profile')
            table.put('user:{}'.format(row.user_id).encode(),
                      {'basic:gender'.encode(): json.dumps(row.gender).encode()})
            table.put('user:{}'.format(row.user_id).encode(),
                      {'basic:birthday'.encode(): json.dumps(age).encode()})
            conn.close()

user_basic.foreachPartition(_udapte_user_basic)
from datetime import datetime
print("{} INFO completely update infomation of basic".format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))